MXNet에서 제공하는 pre-trained word vector를 이용하여 synonyms 탐색 

In [1]:
from mxnet import nd
from mxnet.contrib import text

# pretrained file의 key가 무엇인지 확인
text.embedding.get_pretrained_file_names().keys()

dict_keys(['glove', 'fasttext'])

In [2]:
# glove의 pretrained file 불러오기
print(text.embedding.get_pretrained_file_names('glove'))

['glove.42B.300d.txt', 'glove.6B.50d.txt', 'glove.6B.100d.txt', 'glove.6B.200d.txt', 'glove.6B.300d.txt', 'glove.840B.300d.txt', 'glove.twitter.27B.25d.txt', 'glove.twitter.27B.50d.txt', 'glove.twitter.27B.100d.txt', 'glove.twitter.27B.200d.txt']


In [3]:
# Wikipedia를 통해 pre-training된 50-dimensional GloVe word vector
glove_6b50d = text.embedding.create(
    'glove', pretrained_file_name='glove.6B.50d.txt')

In [5]:
# 총 400001개의 단어를 포함한 word vector
len(glove_6b50d)

400001

In [9]:
# 단어의 인덱스 조회, 인덱스를 통한 단어의 조회
glove_6b50d.token_to_idx['beautiful'], glove_6b50d.idx_to_token[3367]

(3367, 'beautiful')

In [10]:
# k개의 가장 가까운 단어의 거리를 cosine similarity 기준으로 탐색
def knn(W, x, k):
    # The added 1e-9 is for numerical stability
    cos = nd.dot(W, x.reshape((-1,))) / (
        (nd.sum(W * W, axis=1) + 1e-9).sqrt() * nd.sum(x * x).sqrt())
    topk = nd.topk(cos, k=k, ret_typ='indices').asnumpy().astype('int32')
    return topk, [cos[i].asscalar() for i in topk]

In [11]:
# k개의 가장 가까운 단어 추출
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec,
                    embed.get_vecs_by_tokens([query_token]), k+1)
    for i, c in zip(topk[1:], cos[1:]):  # Remove input words
        print('cosine sim=%.3f: %s' % (c, (embed.idx_to_token[i])))

In [13]:
# chip과 가장 가까운 단어 top3
get_similar_tokens('chip', 3, glove_6b50d)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics


In [15]:
# baby와 가장 가까운 단어 top3
get_similar_tokens('baby', 3, glove_6b50d)

cosine sim=0.839: babies
cosine sim=0.800: boy
cosine sim=0.792: girl


In [16]:
get_similar_tokens('beautiful', 3, glove_6b50d)

cosine sim=0.921: lovely
cosine sim=0.893: gorgeous
cosine sim=0.830: wonderful


In [30]:
# 유사 관계 단어 쌍을 알려주고, 나머지 한 단어가 주어졌을 때 그 단어 쌍을 완성시키는 단어 탐색
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed.get_vecs_by_tokens([token_a, token_b, token_c])
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[topk[0]]  # Remove unknown words

In [34]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

In [35]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'

In [36]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'